In [31]:
import pandas as pd
import numpy as np
import math
import datetime

dft = pd.read_csv("times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("complete_distances.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

table=pd.read_csv("route.csv", delimiter=";")
time_table=pd.read_csv("route_time.csv", delimiter=";")

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

In [17]:
"""gained_time1=0
to_2=[]
days = ['martedi', 'mercoledi','giovedì']
#for day in days:
day='martedi'
for i in range(len(time_table[day].dropna())-2):
    try:
        diff=difference(day, i, i+1)
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][i+1]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time1 :
            gained_time1+=(diff-ctime)
            print(f"{day}: primo : {paese} : {gained_time1}")
        else:
            gained_time1+=diff
            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")
            to_2.append(i)
    except:
        print(value)
    #if diff<dft[row[paese].values]:
    #    print("Not acceptable!")"""

'gained_time1=0\nto_2=[]\ndays = [\'martedi\', \'mercoledi\',\'giovedì\']\n#for day in days:\nday=\'martedi\'\nfor i in range(len(time_table[day].dropna())-2):\n    try:\n        diff=difference(day, i, i+1)\n        row=dft[get_row(dft, table[day][i]).values]\n        paese=table[day][i+1]\n        value=row.loc[:,get_column(dft, paese).values]\n        ctime=float(value.values[0][0])\n        #gained_time1=gained_time1+(diff-float(value.values[0][0]))\n        #gained_time2=gained_time2+(diff-float(value.values[0][0]))\n        if (diff+(diff/20))>ctime-gained_time1 :\n            gained_time1+=(diff-ctime)\n            print(f"{day}: primo : {paese} : {gained_time1}")\n        else:\n            gained_time1+=diff\n            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")\n            to_2.append(i)\n    except:\n        print(value)\n    #if diff<dft[row[paese].values]:\n    #    print("Not acceptable!")'

In [18]:
"""impossible=[]
gained_time2=0
for i in range(len(to_2)-1):
    try:
        curr=to_2[i]
        diff=difference(day, curr, to_2[i+1])
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][to_2[i+1]]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time2 :
            gained_time2+=(diff-ctime)
            print(f"{day}: secondo : {paese} : {gained_time2}")
        else:
            print(f"{day}: impossible : {paese} : {diff}  {ctime}")
            impossible.append(i)
    except:
        print(f"-----{value}")"""

'impossible=[]\ngained_time2=0\nfor i in range(len(to_2)-1):\n    try:\n        curr=to_2[i]\n        diff=difference(day, curr, to_2[i+1])\n        row=dft[get_row(dft, table[day][i]).values]\n        paese=table[day][to_2[i+1]]\n        value=row.loc[:,get_column(dft, paese).values]\n        ctime=float(value.values[0][0])\n        #gained_time1=gained_time1+(diff-float(value.values[0][0]))\n        #gained_time2=gained_time2+(diff-float(value.values[0][0]))\n        if (diff+(diff/20))>ctime-gained_time2 :\n            gained_time2+=(diff-ctime)\n            print(f"{day}: secondo : {paese} : {gained_time2}")\n        else:\n            print(f"{day}: impossible : {paese} : {diff}  {ctime}")\n            impossible.append(i)\n    except:\n        print(f"-----{value}")'

In [40]:
#FINAL ALGORITHM (TUE, WED, THU)
start = dft[get_row(dft, 'Cremona, sesto 39').values]
day='giovedi'
#Liste dei due furgoni
f1 = []
f2 = []
#indici
i1 = -1
i2 = -1
#Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
row1 = start
row2 = start
#Lista dei paesi non raggiungibili
impossibile = []
#
for i in range(len(time_table[day].dropna())):
    paese=table[day][i]
    #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
    value1=row1.loc[:,get_column(dft, paese).values]
    value2=row2.loc[:,get_column(dft, paese).values]
    #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
    if(i1 != -1 and i2 != -1):
        diff1=difference(day, i1, i)
        diff2=difference(day, i2, i)
        #print(f"{diff1}, {diff2}")
    else:
        #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
        if(i1 == -1):
            diff1 = value1.values[0][0]
        if(i2 == -1):
            diff2 = value2.values[0][0]
            
    if(diff1 == 0):
        f2.append(i)
        i2 = i
        row2 = dft[get_row(dft, table[day][i]).values]
        continue
    if(diff2 == 0):
        f1.append(i)
        i1 = i
        row1 = dft[get_row(dft, table[day][i]).values]
        continue
        
    #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5 minuti) si sceglie il migliore
  
    if(float(value1.values[0][0])<= float(diff1) + 15 or float(value2.values[0][0])<= float(diff2) + 15):
        if(value1.values[0][0] <= value2.values[0][0]):
            f1.append(i)
            i1 = i
            """print(i)
            if(i >= 13):
                print(f"{diff1}, {diff2}")"""
            row1 = dft[get_row(dft, table[day][i]).values]
        else:
            f2.append(i)
            i2 = i
            row2 = dft[get_row(dft, table[day][i]).values]
    #Impossibilità di raggiungere la locazione in tempo
    else:
        print(f"current time1 {diff1} time {float(value1.values[0][0])} , current time2 {diff2} time {float(value2.values[0][0])}")
        impossibile.append(i)
    
"""print(f1)
print(f2)"""
print(table[day][impossibile])
print(table[day][f1])
print(table[day][f2])

Series([], Name: giovedi, dtype: object)
0               manerbio
1         Cremona, sesto
2          pizzighettone
3       casalpusterlengo
6          pizzighettone
9                  izano
11             offanengo
13    Crema, Civerchi, 9
18         casalmaggiore
Name: giovedi, dtype: object
4     Cremona, Ugolani Dati, 4
5     Cremona, Ugolani Dati, 4
7                    grontardo
8                    pescarolo
10                   vescovato
12                      cingia
14                    solarolo
15                     ostiano
16                  martignana
17              persico dosimo
19                     gadesco
20                     piadena
21              isola dovarese
Name: giovedi, dtype: object


In [13]:
#FINAL ALGORITHM (MON, FRI)
start = dft[get_row(dft, 'Cremona, sesto 39').values]
day='lunedi'
#Liste dei due furgoni
f1 = []

#indici
i1 = -1

#Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
row1 = start

#Lista dei paesi non raggiungibili
impossibile = []
for i in range(len(time_table[day].dropna())-1):
    paese=table[day][i]
    print(paese)
    #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
    value1=row1.loc[:,get_column(dft, paese).values]

    #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
    if(i1 != -1):
        diff1=difference(day, i1, i)
        #print(f"{diff1}, {diff2}")
    else:
        #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
        if(i1 == -1):
            diff1 = value1.values[0][0]
        
    #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5 minuti) si sceglie il migliore
  
    if(float(value1.values[0][0])<= float(diff1)):
        
        f1.append(i)
        i1 = i
        row1 = dft[get_row(dft, table[day][i]).values]
       
    #Impossibilità di raggiungere la locazione in tempo
    else:
        print(f"current time1 {diff1} time {float(value1.values[0][0])}")
        impossibile.append(i)
    
print(f1)

sesto ed
grumello
pizzighettone
MONTODINE
current time1 15.0 time 16.97
ripalta cremasca
CAPRALBA
CASALE CREMASCO
CAMISANO
TRIGOLO
CASTELLEONE
SORESINA
RICENGO
SERGNANO
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12]


0             sesto ed
1             grumello
2        pizzighettone
4     ripalta cremasca
5             CAPRALBA
6      CASALE CREMASCO
7             CAMISANO
8              TRIGOLO
9          CASTELLEONE
10            SORESINA
11             RICENGO
12            SERGNANO
Name: lunedi, dtype: object

In [32]:
def convert(minutes): 
    hour, minu = divmod(minutes, 60) 
    return "%d:%02d" % (hour, minu) 

In [3]:
#FINAL GREEDY ALGORITHM (MON, FRI)
#Single pickup
#paese corrente
curr = 'Cremona, sesto 39'
h = 60*8
#riga distanze corrente
row = dft[get_row(dft, curr).values]
sequence = []
seq_h = []
day = 'venerdi'
for x in range(len(table[day].dropna())):
    #lista delle destinazioni e dei rispettivi tempi di arrivo
    dest = []
    values = []
    for i in range(len(table[day].dropna())):
        #controllo che non sia già stata visitata
        #calcolo le distanze dal paese corrente a tutte le altre destinazioni della giornata
        if(table[day][i] not in sequence):
            dest.append(table[day][i])
            paese = table[day][i]
            values.append(row.loc[:,get_column(dft, paese).values].values[0])
    #aggiorno il paese corrente con quello la cui distanza è minima e lo aggiungo alla sequenza finale
    curr = dest[values.index(min(values))]
    h = h + math.ceil(min(values))
    seq_h.append(convert(h))
    #seq_h.append(math.ceil(min(values)))
    sequence.append(curr)
    #aggiorno la riga
    row = dft[get_row(dft, curr).values]
    
output=zip(sequence, seq_h)
for el in output:
    print(el)

('crema, civerchi', '8:34')
('Capergnanica', '8:46')
('bagnolo', '8:56')
('vaiano', '9:02')
('monte cremasco', '9:06')
('dovera', '9:14')
('pandino', '9:22')
('agnadello', '9:29')
("rivolta d'adda", '9:37')
("spino d'adda", '9:47')
('trescore', '10:02')
('casaletto', '10:07')


In [33]:
#FINAL GREEDY ALGORITHM (TUE, WED, THU)
#Double
#paese corrente
curr = 'Cremona, sesto 39'
h1 = 60*8
h2= 60*8
#riga distanze corrente
row1 = dft[get_row(dft, curr).values]
row2 = dft[get_row(dft, curr).values]
sequence1 = []
seq_h1 = []
sequence2 = []
seq_h2 = []
day = 'giovedi'
for x in range(len(table[day].dropna())):
    #lista delle destinazioni e dei rispettivi tempi di arrivo
    dest1 = []
    values1 = []
    dest2 = []
    values2 = []
    valuesa = []
    valuesb = []
    for i in range(len(table[day].dropna())):
        #controllo che non sia già stata visitata
        #calcolo le distanze dal paese corrente a tutte le altre destinazioni della giornata     
        if(table[day][i] not in sequence1 and table[day][i] not in sequence2):
            dest1.append(table[day][i])
            paese = table[day][i]
            valuesa.append(row1.loc[:,get_column(dft, paese).values].values[0])
            dest2.append(table[day][i])
            paese = table[day][i]
            valuesb.append(row2.loc[:,get_column(dft, paese).values].values[0])
    #aggiorno il paese corrente con quello la cui distanza è minima e lo aggiungo alla sequenza finale
    for k in valuesa:
        values1.append(k[np.argmin(k)])
    for k in valuesb:
        values2.append(k[np.argmin(k)])
    print(values1)
    print(values2)
    try:
        if( min(values1)>=min(values2) ):
            curr1 = dest1[np.argmin(values1)]
            h1 = h1 + math.ceil(values1[np.argmin(values1)])
            seq_h1.append(convert(h1))
            #seq_h.append(math.ceil(min(values)))
            sequence1.append(curr1)
            #aggiorno la riga
            row1 = dft[get_row(dft, curr1).values]
        else:
            curr2 = dest2[np.argmin(values2)]
            h2 = h2 + math.ceil(values2[np.argmin(values2)])
            seq_h2.append(convert(h2))
            #seq_h.append(math.ceil(min(values)))
            sequence2.append(curr2)
            #aggiorno la riga
            row2 = dft[get_row(dft, curr2).values]
    except:
        print(sequence1)
        print(sequence2)
    
output1=zip(sequence1, seq_h1)
output2=zip(sequence2, seq_h2)
print("Furgoncino1:")
for el in output1:
    print(el)
print("\nFurgoncino2:")
for el in output2:
    print(el)

[26.48, 0.17, 19.28, 26.47, 6.67, 6.67, 19.28, 15.45, 17.1, 29.95, 14.05, 34.37, 22.62, 33.33, 27.6, 22.48, 34.43, 8.1, 38.2, 10.72, 26.37, 22.32, 28.48]
[26.48, 0.17, 19.28, 26.47, 6.67, 6.67, 19.28, 15.45, 17.1, 29.95, 14.05, 34.37, 22.62, 33.33, 27.6, 22.48, 34.43, 8.1, 38.2, 10.72, 26.37, 22.32, 28.48]
[26.48, 19.28, 26.47, 6.67, 6.67, 19.28, 15.45, 17.1, 29.95, 14.05, 34.37, 22.62, 33.33, 27.6, 22.48, 34.43, 8.1, 38.2, 10.72, 26.37, 22.32, 28.48]
[26.48, 19.28, 26.47, 6.67, 6.67, 19.28, 15.45, 17.1, 29.95, 14.05, 34.37, 22.62, 33.33, 27.6, 22.48, 34.43, 8.1, 38.2, 10.72, 26.37, 22.32, 28.48]
[27.88, 23.78, 30.68, 0.17, 23.78, 17.58, 18.4, 34.27, 15.23, 38.48, 22.38, 37.5, 27.18, 23.65, 33.97, 10.15, 37.55, 12.0, 27.62, 23.47, 28.02]
[26.48, 19.28, 26.47, 6.67, 19.28, 15.45, 17.1, 29.95, 14.05, 34.37, 22.62, 33.33, 27.6, 22.48, 34.43, 8.1, 38.2, 10.72, 26.37, 22.32, 28.48]
[27.88, 23.78, 30.68, 23.78, 17.58, 18.4, 34.27, 15.23, 38.48, 22.38, 37.5, 27.18, 23.65, 33.97, 10.15, 37.55,

In [26]:
b = [4,5,4,3,3]
np.argmin(b)

3